### 示例8-1

In [ ]:
from pyproj import CRS
from osgeo import gdal
ds = gdal.Open(r"c:\data\landsat\20180523.img")
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
geoTransform = ds.GetGeoTransform()
projection = ds.GetProjection()
print(f"rows:{rows}")
print(f"cols:{cols}")
print(f"bands:{bands}")
print(f"geoTransform:{geoTransform}")
crs = CRS.from_wkt(projection)
print(f"projection:{crs.to_wkt(pretty=True)}")

### 示例8-2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from osgeo import gdal
ds = gdal.Open(r"c:\data\landsat\20180523.img")
band10 = ds.GetRasterBand(10)
histogram = band10.GetDefaultHistogram()
print(f"最小值：{histogram[0]}")
print(f"最大值：{histogram[1]}")
print(f"分级数：{histogram[2]}")
plt.bar(range(histogram[2]),histogram[3])

### 示例8-3

In [ ]:
from osgeo import gdal
ds = gdal.Open(r"c:\data\landsat\20180523.img")
sample_array = ds.ReadAsArray(xoff=100, yoff=100, xsize=5, ysize=5)
print(sample_array)

### 示例8-4

In [ ]:
from osgeo import gdal
def Pixel2world(geotransform, line, column):
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    x = column*pixelWidth + originX - pixelWidth/2
    y = line*pixelHeight + originY - pixelHeight/2
    return(x,y)

#以下代码是读取某个栅格数据，计算其中一个栅格（行为300、列为200）的空间坐标值。
ds = gdal.Open(r"c:\data\landsat\20180523.img")
geotransform = ds.GetGeoTransform()
line = 300;column = 200
x,y = Pixel2world(geotransform, line, column)
print(f"x:{x}")
print(f"y:{y}")

### 示例8-5

In [ ]:
from osgeo import gdal
def world2Pixel(geotransform, x, y):
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    line = int((y-originY)/pixelHeight)+1
    column = int((x-originX)/pixelWidth)+1
    return (line,column)

#以下代码是输入某个空间坐标（x = 359865.0;y = 3468795.0），计算该坐标在栅格数据中的行列号。
ds = gdal.Open(r"c:\data\landsat\20180523.img" )
geotransform = ds.GetGeoTransform()
x = 359880.0;y = 3468780.0
line,column = world2Pixel(geotransform, x, y)
print(f"line:{line}")
print(f"column:{column}")

### 示例8-6

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pyproj import Transformer
from osgeo import gdal
def world2Pixel(geotransform, x, y):
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    line = int((y-originY)/pixelHeight)+1
    column = int((x-originX)/pixelWidth)+1
    return (line,column)

ds = gdal.Open(r"c:\data\landsat\20180523.img")
array = ds.ReadAsArray()
geotransform = ds.GetGeoTransform()
projection = ds.GetProjection()
transformer = Transformer.from_crs(4326,projection,always_xy=True)
#定义样点的土地覆盖类型名称、坐标及光谱曲线的样式
points = [["built_up",121.809,31.14,"-"],
          ["water",121.894,31.107,"--"],
          ["vege",121.606,31.175,":"]]
for point in points:
    x,y = transformer.transform(point[1],point[2])
    line,column = world2Pixel(geotransform, x, y)
    a = array[0:7,line,column]
    plt.plot(a,label=point[0],color="k",linestyle=point[3])
band_label = ["band1","band2","band3","band4","band5","band6","band7"]
plt.xticks([0,1,2,3,4,5,6],band_label)
plt.legend(loc=3)

### 示例8-7

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from osgeo import gdal
ds = gdal.Open( r"c:\data\landsat\20180523.img" )
band1 = ds.GetRasterBand(11)
array = band1.ReadAsArray()
axes_image = plt.imshow(array,cmap=cm.gray)
plt.colorbar(axes_image)

### 示例8-8

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
fig = plt.figure()
ds = gdal.Open(r"c:\data\landsat\20180523.img" )
array = ds.ReadAsArray()
array_543 = array[[4,3,2]].transpose(1,2,0)
#把栅格值转换成0～255之间的整数，但未进行灰度拉伸
vmin = np.min(array_543)
vmax = np.max(array_543)
a = np.array((array_543-vmin)/(vmax-vmin)*255,dtype=int)
ax1 = fig.add_subplot(1,2,1)
plt.imshow(a)
#以均值减2倍标准差和均值加2倍标准差为最小和最大值进行拉伸
mean = np.mean(array_543)
std = np.std(array_543)
vmin=mean-std*2
vmax=mean+std*2
clip = np.clip(array_543,vmin,vmax)
a = np.array((clip-vmin)/(vmax-vmin)*255,dtype=int)
ax2 = fig.add_subplot(1,2,2)
plt.imshow(a)

### 示例8-9

In [ ]:
import fiona
from osgeo import gdal,gdalconst
c = fiona.open(r'c:\data\csj\长三角行政区划.shp', 'r')
LU_x = c.bounds[0]
LU_y = c.bounds[3]
width = c.bounds[2] - c.bounds[0]
height = c.bounds[3] - c.bounds[1]
cols = int(width/1000) + 1
rows = int(height/1000) + 1
projection = c.crs_wkt
driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create(r"c:\data\csj\csj_raster.tif", 
                    xsize=cols,ysize=rows,
                    bands=1,eType=gdalconst.GDT_Byte)
geoTransform = (LU_x, 1000, 0.0, LU_y, 0.0, -1000)
outdata.SetGeoTransform(geoTransform)
outdata.SetProjection(projection)
outdata.FlushCache()
outdata = None

### 示例8-10

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo import gdal,gdalconst
ds = gdal.Open( r"c:\data\landsat\20180523.img" ) 
array = ds.ReadAsArray().astype('f')
NDVI = (array[4] - array[3])/(array[4] + array[3])
driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create(r"c:\data\tmp\NDVI.tif",
                    ds.RasterXSize,ds.RasterYSize,
                    1,gdalconst.GDT_Float32)
outdata.SetGeoTransform(ds.GetGeoTransform())
outdata.SetProjection(ds.GetProjection())
outband=outdata.GetRasterBand(1)  
outband.WriteArray(NDVI)
outdata.FlushCache()
outdata = None

axes_image = plt.imshow(NDVI,cmap=plt.cm.gray)
plt.colorbar(axes_image)

### 示例8-11

In [ ]:
from osgeo import gdal,gdalconst,ogr
in_data =r'c:\data\csj\长三角行政区划.shp'
out_data = r'c:\data\csj\csj_raster_id.tif'
template_data = r"c:\data\csj\csj_raster.tif"
#利用ogr操作矢量数据，返回图层
driver = ogr.GetDriverByName('ESRI Shapefile')
vector_ds = driver.Open(in_data,0)
layer = vector_ds.GetLayer(0)

#获取模板数据参数
template_ds = gdal.Open(template_data)
template_driver = template_ds.GetDriver()
rows = template_ds.RasterYSize
cols = template_ds.RasterXSize
geotransform = template_ds.GetGeoTransform()
projection = template_ds.GetProjection()

dst_driver = template_driver
dst_ds = dst_driver.Create(out_data,cols,rows,1,gdalconst.GDT_Byte)
dst_ds.SetGeoTransform(geotransform)
dst_ds.SetProjection(projection)
gdal.RasterizeLayer(dst_ds,[1],layer,options=["ATTRIBUTE=OBJECTID"])
dst_ds.FlushCache()
dst_ds = None

#利用matplotlib显示
import matplotlib.pyplot as plt
dst_ds = gdal.Open(out_data)
array = dst_ds.ReadAsArray()
axes_image = plt.imshow(array,cmap=plt.cm.gray)
plt.colorbar(axes_image)

### 示例8-12

In [ ]:
from osgeo import gdal
in_data = r'c:\data\csj\csj_raster_id.tif'
ds = gdal.Open(in_data)
band = ds.GetRasterBand(1)
rat = gdal.RasterAttributeTable()
#添加三个字段，分别表示像元值、对应的名称和像元数
rat.CreateColumn("Value",gdal.GFT_Integer,gdal.GFU_Name)
rat.CreateColumn("Name",gdal.GFT_String,gdal.GFU_Generic)
rat.CreateColumn("PixelCount",gdal.GFT_Integer,gdal.GFU_PixelCount)
codes = {0:"其它区域",
1:"安庆市",2:"池州市",3:"滁州市",4:"合肥市",5:"马鞍山市",6:"铜陵市",
7:"芜湖市",8:"宣城市",9:"常州市",10:"南京市",11:"南通市",12:"苏州市",
13:"泰州市",14:"无锡市",15:"盐城市",16:"扬州市",17:"镇江市",18:"上海市",
19:"杭州市",20:"湖州市",21:"嘉兴市",22:"金华市",23:"宁波市",24:"绍兴市",
25:"台州市",26:"舟山市"}

histogram = band.GetDefaultHistogram()
count = len(codes)
rat.SetRowCount(count)
i = 0
#以下循环中i表示属性表的行，0、1、2表示属性表的列
for code in codes:
    rat.SetValueAsString(i,0,code)
    rat.SetValueAsString(i,1,codes[code])
    rat.SetValueAsInt(i,2,histogram[3][i])
    i = i + 1

band.SetDefaultRAT(rat)
ds.FlushCache()
del band,ds

#以下代码是读取记录在栅格数据中的属性表信息（栅格值、对应的行政单元名和栅格数）：
in_data = r'c:\data\csj\csj_raster_id.tif'
ds = gdal.Open(out_data)
band = ds.GetRasterBand(1)
rat = band.GetDefaultRAT()
array0 = rat.ReadAsArray(0)
array1 = rat.ReadAsArray(1)
array2 = rat.ReadAsArray(2)
print("代码 名称 像元数")
for i in range(len(array0)):
    print(array0[i],array1[i].decode(encoding="utf8"),array2[i])

### 示例8-13

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from osgeo import gdal
import numpy as np
import fiona

#行列坐标转地理坐标
def Pixel2world(geotransform, line, column):
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    x = column*pixelWidth + originX - pixelWidth/2
    y = line*pixelHeight + originY - pixelHeight/2
    return(x,y)

#IDW插值函数（返回指定栅格的插值结果）
def idw(x,y,c,value_field):
    """
    x,y:栅格的x、y坐标
    c:点数据的Collection对象
    value_field:点数据的插值字段
    """
    i_dist2_list = []
    value_list = []
    for record in c:
        #返回离散点的x和y坐标
        sr_x = record['geometry']['coordinates'][0]
        sr_y = record['geometry']['coordinates'][1]
        #计算栅格点与离散点距离平方倒数并追加到列表
        i_dist2 = 1/((x-sr_x)**2 + (y-sr_y)**2)
        i_dist2_list.append(i_dist2)
        #计算离散点的内插字段值并追加到列表
        value = record['properties'][f'{value_field}']
        value_list.append(value) 
    i_dist2_sum = sum(i_dist2_list)
    idw_value = 0
    for i in range(len(i_dist2_list)):
        idw_value = i_dist2_list[i]/i_dist2_sum*value_list[i] + idw_value
    return idw_value

#定义栅格的大小为500m×500m，可以改变栅格大小
x_resolution = 500
y_resolution = 500
c = fiona.open(r'c:\data\csj\pm10_pts.shp', 'r')
value_field = "pm10"
crs = c.crs_wkt
xmin = c.bounds[0];ymin = c.bounds[1]
xmax = c.bounds[2];ymax = c.bounds[3]
xsize = int((xmax-xmin)/x_resolution) + 1
ysize = int((ymax-ymin)/y_resolution) + 1
geoTransform = (xmin,y_resolution,0,ymax,0,-y_resolution)
a = np.zeros((ysize,xsize))
#计算每个栅格的值
for i in range(ysize):
    for j in range(xsize):
        x,y = Pixel2world(geoTransform,i,j)
        idw_value = idw(x,y,c,value_field)
        a[i][j] = idw_value
driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create(r"c:\data\tmp\new_image.tif",
                   xsize=xsize,ysize=ysize,bands=1)
outband=outdata.GetRasterBand(1)  
outband.WriteArray(a.astype(int))
outdata.SetGeoTransform(geoTransform)
outdata.SetProjection(crs)
outdata.FlushCache()
outdata = None
img = plt.imread(r"c:\data\tmp\new_image.tif")
axes_image = plt.imshow(img,cmap=cm.gray)
plt.colorbar(axes_image)

### 示例8-14

In [ ]:
def sub_image(in_raster,out_raster,bound,bound_coor_system):
    """
    提取栅格数据子区。
    in_raster：输入栅格数据
    out_raster：输出栅格数据
    bound：子区范围（左下角和右上角坐标）
    bound_coor_system：子区范围坐标系统
    """
    from pyproj import Transformer
    from osgeo import gdal

    #空间坐标转行列坐标
    def world2Pixel(geotransform, x, y):
        originX = geotransform[0]
        originY = geotransform[3]
        pixelWidth = geotransform[1]
        pixelHeight = geotransform[5]
        line = int((y-originY)/pixelHeight)+1
        column = int((x-originX)/pixelWidth)+1
        return (line,column)
    
    src_ds = gdal.Open(in_raster)
    src_driver = src_ds.GetDriver()
    src_bands = src_ds.RasterCount
    src_dataType = src_ds.GetRasterBand(1).DataType
    src_geotransform = src_ds.GetGeoTransform()
    src_projection = src_ds.GetProjection()
    
    #子区左下角和右上角经纬度坐标转输入栅格数据的坐标系统，然后返回行列坐标
    minx,miny,maxx,maxy = bound
    LL = (minx,miny)
    UR = (maxx,maxy)
    transformer = Transformer.from_crs(bound_coor_system,src_projection,always_xy=True)
    LL_x,LL_y = transformer.transform(LL[0],LL[1])
    UR_x,UR_y = transformer.transform(UR[0],UR[1])
    LL_line,LL_column = world2Pixel(src_geotransform, LL_x, LL_y)
    UR_line,UR_column = world2Pixel(src_geotransform, UR_x, UR_y)
    
    #读子区数据
    array = src_ds.ReadAsArray(xoff=LL_column,yoff=UR_line,
                           xsize=UR_column-LL_column,
                           ysize=LL_line-UR_line)
    
    #如输入栅格数据是单波段数据，把二维数组设置成三维数组
    if src_bands == 1:
        array.shape = (1,LL_line-UR_line,UR_column-LL_column)

    #创建输出栅格数据，其中driver、bands、eType、Projection同输入栅格数据
    dst_driver = src_driver
    dst_ds = dst_driver.Create(out_raster,
                          UR_column-LL_column,LL_line-UR_line,
                          src_bands,src_dataType)
    dst_ds.SetProjection(src_projection)
    
    #设置新数据的坐标转换参数（修改左上角栅格的x和y坐标）
    new_gt = list(src_geotransform)
    new_gt[0] = LL_x
    new_gt[3] = UR_y
    dst_ds.SetGeoTransform(new_gt)
    
    #array写入到栅格数据中
    for i in range(src_bands):
        band = dst_ds.GetRasterBand(i+1)
        band.WriteArray(array[i])
    dst_ds.FlushCache()
    dst_ds = None

#以下代码是调用函数从20180523.img栅格数据中提取出一个子区：
in_raster = r"c:\data\landsat\20180523.img"
out_raster = r"c:\data\tmp\sub_image.img"
bound = (121.73,31.09,121.88,31.21)
crs = 4326
sub_image(in_raster,out_raster,bound,crs)

### 示例8-15

In [ ]:
def bounds(in_raster,out_coor_system):
    """
    计算栅格数据坐标转换后的矩形范围（返回最小最大x和y坐标）
    in_raster：输入栅格数据
    out_coor_system：输出栅格数据坐标系统
    """
    from pyproj import Transformer
    from osgeo import gdal
    x_ts = [];y_ts = []    #边缘栅格坐标转换后的x和y坐标列表

    #获取输入栅格数据信息
    ds = gdal.Open(in_raster)
    lines = ds.RasterYSize 
    cols = ds.RasterXSize
    geotransform = ds.GetGeoTransform()
    projection = ds.GetProjection()
    #对每个边缘栅格的x、y坐标进行转换
    transformer = Transformer.from_crs(projection,out_coor_system,always_xy=True)
    for line in range(lines):
        #如果是第一行或最后一行，则计算每一列栅格
        if line == 0 or line == lines-1:
            for col in range(cols):
                x,y = Pixel2world(geotransform,line+1,col+1)
                x_t,y_t = transformer.transform(x,y)
                x_ts.append(x_t)
                y_ts.append(y_t)
        #如果不是第一行或最后一行，只计算第一列和最后一列栅格
        else:
            x,y = Pixel2world(geotransform, line+1,1)
            x_t,y_t = transformer.transform(x,y)
            x_ts.append(x_t)
            y_ts.append(y_t)
            x,y = Pixel2world(geotransform, line+1,cols)
            x_t,y_t = transformer.transform(x,y)
            x_ts.append(x_t)
            y_ts.append(y_t)
    #返回最小、最大x、y坐标
    x_t_min = round(min(x_ts),2)
    y_t_min = round(min(y_ts),2)
    x_t_max = round(max(x_ts),2)
    y_t_max = round(max(y_ts),2)    
    return (x_t_min,y_t_min,x_t_max,y_t_max)

#以下代码是计算20180523.img栅格数据坐标系统转换（从UTM_Zone_51N坐标转换为Albers等积投影坐标）后的矩形范围：
from pyproj import CRS
from pyproj.enums import WktVersion
in_raster = r"c:\data\landsat\20180523.img" 
aea = "+proj=aea +lat_1=25 +lat_2=45 +lat_0=0 +lon_0=105"
crs = CRS.from_proj4(aea)
aea_wkt = crs.to_wkt(version=WktVersion.WKT1_GDAL,pretty=True)
bounds(in_raster,aea_wkt)


### 示例8-16

In [ ]:
def projectRaster(in_raster,out_raster,out_coor_system,x_size=None,y_size=None):
    """
    栅格数据坐标转换
    in_raster：输入栅格数据
    out_raster：输出栅格数据
    out_coor_system：输出栅格数据的坐标系统
    x_size，y_size：输出栅格x和y方向的大小，如未定义则按行列数和范围计算
    """
    from pyproj import Transformer
    from shapely.geometry import Polygon
    from shapely.ops import transform
    from osgeo import gdal
    from osgeo import gdal,gdalconst
    #获取输入栅格数据信息
    src_ds = gdal.Open(in_raster)
    src_driver = src_ds.GetDriver()
    src_lines = src_ds.RasterYSize 
    src_cols = src_ds.RasterXSize
    src_bands = src_ds.RasterCount
    src_dataType = src_ds.GetRasterBand(1).DataType 
    src_geotransform = src_ds.GetGeoTransform()
    src_projection = src_ds.GetProjection()
    
    #调用bounds()函数
    minx,miny,maxx,maxy = bounds(in_raster,out_coor_system)
   
    #计算输出栅格数据的行列数
    if y_size != None:
        dst_lines = int((maxy - miny)/y_size)
    else:
        dst_lines = src_lines
        y_size = (maxy - miny)/src_lines
    
    if x_size != None:
        dst_cols = int((maxx - minx)/x_size)
    else:
        dst_cols = src_cols
        x_size = (maxx - minx)/src_cols
    
    #创建输出栅格数据，其中driver、bands、eType同输入栅格数据
    dst_driver = src_driver
    dst_ds = dst_driver.Create(out_raster,
                          xsize=dst_cols,ysize=dst_lines,
                          bands = src_bands,
                          eType = src_dataType)
    dst_GeoTransform = (minx,x_size,0,maxy,0,-y_size)
    dst_ds.SetGeoTransform(dst_GeoTransform)
    dst_ds.SetProjection(out_coor_system)
    
    #对输出栅格数据进行重采样，并写到文件中
    gdal.Warp(dst_ds,src_ds)
    dst_ds.FlushCache()
    dst_ds = None

#以下代码是把20180523.img栅格数据的坐标系统从UTM_Zone_51N转换为Albers等积投影坐标。
in_raster = r"c:\data\landsat\20180523.img"
out_raster = r"c:\data\landsat\20180523_aea.img"
aea = "+proj=aea +lat_1=25 +lat_2=45 +lat_0=0 +lon_0=105"
crs = CRS.from_proj4(aea)
aea_wkt = crs.to_wkt(version=WktVersion.WKT1_GDAL,pretty=True)
projectRaster(in_raster,out_raster,aea_wkt,x_size=30,y_size=30)


### 示例8-17

In [ ]:
def mask_image(in_raster,out_raster,in_vector):
    """
    对栅格数据进行掩膜处理。本函数没有考虑坐标转换，
    要求输入的栅格数据和矢量数据的坐标系统一致。
    in_raster：输入栅格数据
    out_raster：掩膜处理后的栅格数据
    in_vector：用于掩膜的矢量数据，shapefile数据
    """
    from osgeo import gdal,gdalconst,ogr
   
    #使用ogr读矢量数据
    driver = ogr.GetDriverByName('ESRI Shapefile')
    vector_ds = driver.Open(in_vector,0)
    layer = vector_ds.GetLayer(0)

    #获取输入栅格数据参数
    in_ds = gdal.Open(in_raster)
    in_driver = in_ds.GetDriver()
    rows = in_ds.RasterYSize
    cols = in_ds.RasterXSize
    bands = in_ds.RasterCount
    geotransform = in_ds.GetGeoTransform()
    projection = in_ds.GetProjection()
    
    #矢量数据转栅格数据
    mask_raster = r"c:\data\tmp\tmp_mask.img"
    mask_ds = in_driver.Create(mask_raster,cols,rows,1,gdalconst.GDT_Byte)
    mask_ds.SetGeoTransform(geotransform)
    mask_ds.SetProjection(projection)
    gdal.RasterizeLayer(mask_ds,[1],layer)
    mask_array = mask_ds.ReadAsArray()
   
    #复制输入数据，把矢量数据范围外的栅格赋值为0，并设为空值
    out_ds = in_driver.CreateCopy(out_raster,in_ds)
    for i in range(bands):
        band = out_ds.GetRasterBand(i+1)
        out_array = band.ReadAsArray()
        out_array[mask_array == 0] = 0
        band.WriteArray(out_array)
        band.SetNoDataValue(0)
    out_ds.FlushCache()
    out_ds = None

#以下代码是对20180523_aea.img栅格数据进行掩膜处理，用于掩膜的矢量数据是mask.shp。
in_raster = r"c:\data\landsat\20180523_aea.img"
out_raster = r'c:\data\landsat\20180523_mask.img'
in_vector = r'c:\data\landsat\mask.shp'
mask_image(in_raster,out_raster,in_vector)

### 示例8-18

In [ ]:
%matplotlib qt
import numpy as np
import cv2 as cv
from osgeo import gdal
from matplotlib import pyplot as plt
ds = gdal.Open(r"c:\data\landsat\20180523.img" )
band1 = ds.GetRasterBand(11)
img = band1.ReadAsArray()

def stretch(img):
    mean = np.mean(img)
    std = np.std(img)
    vmin=mean-std*2
    vmax=mean+std*2
    clip = np.clip(img,vmin,vmax)
    stretch = np.array((clip-vmin)/(vmax-vmin)*255,dtype=int)
    return stretch

laplacian = cv.Laplacian(img,cv.CV_64F)
sobelx = cv.Sobel(img,cv.CV_64F,1,0,ksize=9)
sobely = cv.Sobel(img,cv.CV_64F,0,1,ksize=9)
plt.rcParams['xtick.bottom'] = False
plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.labelleft'] = False
plt.subplot(2,2,1)
plt.imshow(stretch(img),cmap = 'gray')
plt.title('Original')
plt.subplot(2,2,2)
plt.imshow(stretch(laplacian),cmap = 'gray')
plt.title('Laplacian')
plt.subplot(2,2,3)
plt.imshow(stretch(sobelx),cmap = 'gray')
plt.title('Sobel X')
plt.subplot(2,2,4)
plt.imshow(stretch(sobely),cmap = 'gray')
plt.title('Sobel Y')
plt.show()

### 示例8-19

In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from sklearn.cluster import KMeans
ds = gdal.Open(r"c:\data\landsat\20180523.img" )
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
array = ds.ReadAsArray()
#先转成二维数组（bands*samples），然后再转置成samples*bands
array = array.reshape(bands,-1).transpose()  
kMeans = KMeans(n_clusters=3, random_state=0)
cluster = kMeans.fit_predict(array)
#一维数组通过reshape转成二维数组（rows*columns）
cluster.shape = (rows,cols)
plt.axis("off")
plt.imshow(cluster)